In [1]:
suppressWarnings({suppressMessages({is_installed <- require('kerastuneR')})})

if(!is_installed){
    install.packages('kerastuneR')
}

In [2]:
library(keras)
library(rsample)
library(tensorflow)
library(kerastuneR)

In [5]:
set.seed(1)

willsee <- caret::createDataPartition(iris[['Sepal.Width']],list = FALSE,p = .6)

train <- iris[willsee,][,-5]
test <- iris[-willsee,][,-5]


recete <- Sepal.Width ~ .

tarif <- recipes::recipe(recete,train) %>%
recipes::step_normalize(recipes::all_numeric_predictors())

train <- tarif %>% recipes::prep() %>% recipes::bake(new_data=train)
test <- tarif %>% recipes::prep() %>% recipes::bake(new_data=test)

train_x <- train[,-4] %>% as.matrix
test_x <- test[,-4] %>% as.matrix

train_y <- train[,4] %>% as.matrix
test_y <- test[,4] %>% as.matrix

In [8]:
hp = HyperParameters()
hp$Choice('learning_rate', c(1e-1, 1e-3,1e-4))
hp$Int('num_layers', 2L, 5L)


mnist_model = function(hp) {
  
  model = keras_model_sequential() %>% 
    layer_flatten(input_shape = ncol(train_x))
  for (i in 1:(hp$get('num_layers')) ) {
    model %>% layer_dense(32, activation='relu') %>% 
      layer_dense(units = 1, activation='linear')
  } %>% 
    compile(
      optimizer = tf$keras$optimizers$Adam(hp$get('learning_rate')),
      loss = 'mse',
      metrics = 'mean_absolute_error') 
  return(model)
  
}


[1] 0.1

[1] 2

In [13]:
tuner <- RandomSearch(
  mnist_model,
  hyperparameters = hp,
  objective = 'val_mean_absolute_error',
  max_trials = 5,
  executions_per_trial = 3,
  directory = 'my_dir4',
  project_name = 'bb')

In [14]:
tuner %>% search_summary()
 

tuner %>% fit_tuner(train_x,train_y,
                    epochs=10, 
                    validation_data = list(test_x,test_y))

In [15]:
modk <- tuner %>% get_best_models(1) %>% .[[1]]

In [16]:
modk2 <- lm(formula = Sepal.Width ~ .,data = train)

character(0)

In [22]:
tuner$get_best_hyperparameters

<bound method BaseTuner.get_best_hyperparameters of <keras_tuner.tuners.randomsearch.RandomSearch object at 0x0000021F16AECB50>>